In [93]:
import torch
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 197376  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 131584  
4  | bottleneck.3 | ReLU             | 0       
5  | decoder1     | Sequential       | 8389376 
6  | decoder1.0   | ConvTranspose3d  | 8388864 
7  | decoder1.1   | BatchNorm3d      | 512     
8  | decoder1.2   | ReLU             | 0       
9  | decoder2     | Sequential       | 2097536 
10 | decoder2.0   | ConvTranspose3d  | 2097280 
11 | decoder2.1   | BatchNorm3d      | 256     
12 | decoder2.2   | ReLU             | 0       
13 | decoder3     | Sequential       | 524480  
14 | decoder3.0   | ConvTranspose3d  | 524352  
15 | decoder3.1   | BatchNorm3d      | 128     
16 | decoder3.2   | ReLU             | 0       
17 | decoder4     | Sequential       | 4097    
18 | decoder4.0   | ConvTranspose3

In [95]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train', filter_class='watercraft')
val_dataset = ShapeNet('val', filter_class='watercraft')
test_dataset = ShapeNet('test', filter_class='watercraft')

print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
print(f'Length of val set: {len(val_dataset)}')  # expected output: 153540
print(f'Length of test set: {len(test_dataset)}')  # expected output: 64

Length of train set: 1551
Length of val set: 194
Length of test set: 194


In [82]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = val_dataset[0]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)
print(f'Target DF: {type(sample["target_df"])}')  # expected output: <class 'numpy.ndarray'>

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)
Target DF: <class 'numpy.ndarray'>


Output()

In [ ]:
##################
#                #
#    TRAINING    #
#                #
##################

In [ ]:
# OVERFIT
from training import train
config = {
    'experiment_name': 'overfit',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 4,
    'print_every_n': 1,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': False,
    'kl_weight': 1,
    'resume_ckpt': None

}
train.main(config)

In [ ]:
# AIRPLANE VAD
from training import train
config = {
    'experiment_name': 'airplane_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': 'airplane_vad',
    'filter_class': 'airplane',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# AIRPLANE AD
from training import train
config = {
    'experiment_name': 'airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 20,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'airplane',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# CHAIR
from training import train
config = {
    'experiment_name': 'chair_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'chair',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# SOFA & CHAIR VAD
from training import train
config = {
    'experiment_name': 'sofa_chair_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa_chair',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# SOFA VAD
from training import train
config = {
    'experiment_name': 'sofa_vad_0_03kl',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# SOFA AD
from training import train
config = {
    'experiment_name': 'sofa_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.01,
    'resume_ckpt': None,
    'filter_class': 'sofa',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# SOFA VAD
from training import train
config = {
    'experiment_name': 'car_vad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : True,
    'test': False,
    'kl_weight': 0.03,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : True
}
train.main(config)

In [ ]:
# CAR AD
from training import train
config = {
    'experiment_name': 'car_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': None,
    'filter_class': 'car',
    'decoder_var' : False
}
train.main(config)

In [ ]:
# CAR AIRPLANE AD
from training import train
config = {
    'experiment_name': 'car_airplane_ad',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 64,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 500,
    'print_every_n': 100,
    'latent_code_length' : 256,
    'scheduler_step_size': 100,
    'vad_free' : False,
    'test': False,
    'kl_weight': 0.0,
    'resume_ckpt': 'car_airplane_ad',
    'filter_class': 'car_airplane',
    'decoder_var' : False
}
train.main(config)

In [38]:
#####################
#                   #
#    VISUALIZING    #
#                   #
#####################

In [39]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch

def visualize_dataset_sample(filter_class, index):
    dataset = ShapeNet('train', filter_class = filter_class)
    sample = dataset[index]
    input_mesh = marching_cubes(sample['target_df'], level=1)
    visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

def visualize_ad(experiment, index):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x = latent_vectors[index].unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_vad(experiment, index):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    log_vars = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x = latent_vectors[index]
    Dist = dist.Normal(x, torch.exp(log_vars[index]))
    x_vad = Dist.rsample().unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x_vad)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_vad_norm(experiment):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Sample
    # Dist = dist.Normal(torch.zeros(256), torch.ones(256))
    x_vad = torch.randn(256).unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x_vad)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

def visualize_interpolation_ad(experiment, index1, index2, a1=0.5, a2=0.5):
    # Load model
    model = ThreeDEPNDecoder()
    model.load_state_dict(torch.load(f"runs/{experiment}/model_best.ckpt", map_location='cpu'))
    # Load latent codes
    latent_vectors = torch.load(f"runs/{experiment}/latent_best.pt", map_location = 'cpu')
    # Sample
    x1 = latent_vectors[index1]
    x2 = latent_vectors[index2]
    x = (a1*x1 + a2*x2).unsqueeze(0)
    # Forward pass
    output_meshes_int = model(x)
    # Visualize
    output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
    visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

In [61]:
import random
experiment = "sofa_vad"
experiment2 = "sofa_ad"
# experiment2 = "sofa_ad"
filter_class = "car_airplane"
index = 4123
index1 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
index2 = random.choice(range(len(ShapeNet('train', filter_class = filter_class))))
a1 = 0.5
a2 = 1 - a1
#-------
# visualize_vad_norm(experiment2)
visualize_vad_norm(experiment)
visualize_vad_norm(experiment2)
# visualize_ad(experiment, index)
#-------
# visualize_vad_norm(experiment)
# visualize_vad_norm(experiment2)
# visualize_ad(experiment2, index)
# visualize_dataset_sample(filter_class, index)
#-------
# visualize_interpolation_ad(experiment, index1, index2, a1, a2)
# visualize_ad(experiment, index1)
# visualize_ad(experiment, index2)

Output()

Output()

In [ ]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch

# load model
name_temp = 'sofa_ad'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')
Dist = dist.Normal(latent_vectors[1002], torch.exp(log_vars[1002]))
Dist2 = dist.Normal(latent_vectors[1000], torch.exp(log_vars[1000]))
#x_vad_n = torch.randn(256, device='cpu')
x_vad = Dist.rsample()#.unsqueeze(0)
x_vad2 = Dist2.rsample()#.unsqueeze(0)

a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((x_vad*a1 + x_vad2*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)

output_meshes = model(x_vad.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(x_vad2.unsqueeze(0))
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)


In [ ]:
from model.threedepn import ThreeDEPNDecoder
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
import torch
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[5]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)
# load model
name_temp = 'generalization_airplane_no_vad'
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{name_temp}/model_best.ckpt", map_location='cpu'))

# load latent codes and latent variances
latent_vectors = torch.load(f"runs/{name_temp}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{name_temp}/log_var_best.pt", map_location = 'cpu')

x_vad_n = x_vad_n.unsqueeze(0)
output_meshes = model(latent_vectors[5].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

output_meshes = model(latent_vectors[105].unsqueeze(0))

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)
a1 = 0.5
a2 = 0.5
f1 = 5
f2 = 105
output_meshes_int = model((latent_vectors[f1]*a1 + latent_vectors[f2]*a2).unsqueeze(0))
output_mesh_int = marching_cubes(output_meshes_int[0].detach().numpy(), level=1)
visualize_mesh(output_mesh_int[0], output_mesh_int[1], flip_axes=True)